In [28]:
#pip install selenium

In [29]:
#pip install mysql-connector-python

In [30]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from datetime import date, datetime

driver = webdriver.Chrome()

show_result = True

bus_details_extracted = []
xpath_info = {"bus_name":[1,1],"bus_type":[1,2],"depart_time":[2,1],"duration":[3,0],"react_time":[4,1],"rating":[5,1],"price":[6,0],"availability":[7,1]}

# Extract bus details for the given route
def extract_bus_details(buses_details, route_name, route_link):
    for i in range(1, 100):
        single_bus_detail = [route_name, route_link]
        for x in xpath_info.values():
            row = x[0]
            col = x[1]
            xpath_string = "/html/body/section/div[2]/div[4]/div/div[2]/div/div[2]/div[2]/div[3]/ul/div[" + str(i) + "]/li/div/div[1]/div[1]/div[" + str(row) + "]/div"
            if col > 0:
                xpath_string += ("[" + str(col) + "]")
            try:
                detail_element = buses_details.find_element(By.XPATH, xpath_string).text
                if row == 6:
                    detail_element = (detail_element.split(" "))[-1]
                if row == 7:
                    detail_element = (detail_element.split(" "))[0]
                if row ==2 and col ==1:
                    detail_element = str(date.today()) + (" " + detail_element)
                if row == 4 and col == 1:
                    arriv_date = ""
                    try:
                        date_xpath = "/html/body/section/div[2]/div[4]/div/div[2]/div/div[2]/div[2]/div[3]/ul/div[1]/li/div/div[1]/div[1]/div[4]/div[3]"
                        buses_details.find_element(By.XPATH, date_xpath)
                        arriv_date = str(tomorrow = date.today() + timedelta(days=1)) + (" " + detail_element)
                    except Exception as e:
                        arriv_date = str(date.today()) + (" " + detail_element)
                    detail_element = arriv_date
                single_bus_detail.append(detail_element)
            except Exception as e:
                if not "Unable to locate element" in str(e):
                    print(e)
                #single_bus_detail.append(None)
                break
        if len(single_bus_detail) == 10:
            bus_details_extracted.append(single_bus_detail)


# Function to extract bus routes and links from the current page
def extract_routes(state_transport_url):

    driver.get(state_transport_url)
    time.sleep(3)

    routes = driver.find_elements(By.CLASS_NAME,"route_details")
    row = 1
    for route in routes:
        driver.execute_script("window.scrollBy(0, "+ str(120*row) +");")
        time.sleep(3)

        route_link_xpath = "/html/body/div[1]/div/div[4]/div[" + str(row+1) + "]/div[1]/a"
        route_link_element = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, route_link_xpath)))
        route_link_element = driver.find_element(By.XPATH, route_link_xpath)
        
        route_name = route_link_element.get_attribute('title')
        route_link = route_link_element.get_attribute('href')
        print(route_name)
        
        route_link_element.click()
        time.sleep(5)




        

        for i in range(5):
            driver.execute_script("window.scrollBy(0, 5000);")
            time.sleep(3)

        buses_details = driver.find_element(By.CLASS_NAME,"bus-items")
        extract_bus_details(buses_details, route_name, route_link)

        row += 1
        driver.get(state_transport_url)
        time.sleep(5)
        

# Extract data from the pages
def extract_data():
    driver.get('https://www.redbus.in/')
    driver.maximize_window()
    time.sleep(2)
    driver.execute_script("window.scrollBy(0, 800);")

    state_transport_url_list = []
    # Extract state transport URLs
    link_element = driver.find_element(By.XPATH, "/html/body/section/div[2]/main/div[4]/div[3]/section/div/div[2]/div[1]/div[8]/div[2]/span/p")
    state_transport_link_element = link_element.find_elements(By.TAG_NAME, "a");

    for state_transport_link in state_transport_link_element:
        state_transport_url_list.append(state_transport_link.get_attribute('href'))

    for state_transport_url in state_transport_url_list:
        try:
            extract_routes(state_transport_url)
        
        except Exception as e:
            print("Couldn't extract the routes,e")
        

# Main Function
def main():
    try:
        extract_data()
    except Exception as e:
        print("Exception while extracting data", e)
    if show_result:
        print(bus_details_extracted)
    driver.quit()

#Initial main() call
main()

Vijayawada to Hyderabad
[['Vijayawada to Hyderabad', 'https://www.redbus.in/bus-tickets/vijayawada-to-hyderabad', 'IntrCity SmartBus', 'A/C Seater / Sleeper (2+1)', '2024-07-18 23:45', '06h 15m', '2024-07-18 06:00', '4.6', '477', '40'], ['Vijayawada to Hyderabad', 'https://www.redbus.in/bus-tickets/vijayawada-to-hyderabad', 'IntrCity SmartBus', 'A/C Seater / Sleeper (2+1)', '2024-07-18 23:25', '07h 00m', '2024-07-18 06:25', '4.6', '571', '37'], ['Vijayawada to Hyderabad', 'https://www.redbus.in/bus-tickets/vijayawada-to-hyderabad', 'FRESHBUS', 'Electric A/C Seater (2+2)', '2024-07-18 22:30', '06h 50m', '2024-07-18 05:20', '4.6', '450', '29'], ['Vijayawada to Hyderabad', 'https://www.redbus.in/bus-tickets/vijayawada-to-hyderabad', 'AdSai RK Travels', 'Non A/C Seater / Sleeper (2+1)', '2024-07-18 22:10', '07h 20m', '2024-07-18 05:30', '4.0', '519', '28'], ['Vijayawada to Hyderabad', 'https://www.redbus.in/bus-tickets/vijayawada-to-hyderabad', 'FRESHBUS', 'Electric A/C Seater (2+2)', '202

In [31]:
import mysql.connector

db = mysql.connector.connect(user="root", password = "poppy#558",database="red_bus",auth_plugin='mysql_native_password')
cursor = db.cursor()

add_bus_details = """INSERT INTO bus_routes
                   (route_name, route_link, bus_name, bus_type, departing_time, duration, reaching_time, star_rating, price, seats_available)
                   VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"""

#bus_details_extracted = [['Vijayawada to Hyderabad', 'https://www.redbus.in/bus-tickets/vijayawada-to-hyderabad', 'IntrCity SmartBus', 'A/C Seater / Sleeper (2+1)', '2024-07-18 23:45', '06h 15m', '2024-07-18 06:00', '4.6', '524', '40'], ['Vijayawada to Hyderabad', 'https://www.redbus.in/bus-tickets/vijayawada-to-hyderabad', 'IntrCity SmartBus', 'A/C Seater / Sleeper (2+1)', '2024-07-18 23:25', '07h 00m', '2024-07-18 06:25', '4.6', '571', '37'], ['Vijayawada to Hyderabad', 'https://www.redbus.in/bus-tickets/vijayawada-to-hyderabad', 'FRESHBUS', 'Electric A/C Seater (2+2)', '2024-07-18 22:30', '06h 50m', '2024-07-18 05:20', '4.6', '450', '29'], ['Vijayawada to Hyderabad', 'https://www.redbus.in/bus-tickets/vijayawada-to-hyderabad', 'AdSai RK Travels', 'Non A/C Seater / Sleeper (2+1)', '2024-07-18 22:10', '07h 20m', '2024-07-18 05:30', '4.0', '519', '28'], ['Vijayawada to Hyderabad', 'https://www.redbus.in/bus-tickets/vijayawada-to-hyderabad', 'FRESHBUS', 'Electric A/C Seater (2+2)', '2024-07-18 05:00', '07h 05m', '2024-07-18 12:05', '4.7', '480', '29'], ['Vijayawada to Hyderabad', 'https://www.redbus.in/bus-tickets/vijayawada-to-hyderabad', 'FRESHBUS', 'Electric A/C Seater (2+2)', '2024-07-18 11:40', '06h 55m', '2024-07-18 18:35', '4.7', '450', '38'], ['Vijayawada to Hyderabad', 'https://www.redbus.in/bus-tickets/vijayawada-to-hyderabad', 'FRESHBUS', 'Electric A/C Seater (2+2)', '2024-07-18 17:30', '06h 25m', '2024-07-18 23:55', '4.6', '450', '39'], ['Vijayawada to Hyderabad', 'https://www.redbus.in/bus-tickets/vijayawada-to-hyderabad', 'FRESHBUS', 'Electric A/C Seater (2+2)', '2024-07-18 16:20', '06h 35m', '2024-07-18 22:55', '4.6', '450', '42'], ['Vijayawada to Hyderabad', 'https://www.redbus.in/bus-tickets/vijayawada-to-hyderabad', 'FRESHBUS', 'Electric A/C Seater (2+2)', '2024-07-18 21:30', '06h 35m', '2024-07-18 04:05', '4.5', '450', '32'], ['Vijayawada to Hyderabad', 'https://www.redbus.in/bus-tickets/vijayawada-to-hyderabad', 'IntrCity SmartBus', 'AC Sleeper (2+1)', '2024-07-18 23:40', '06h 50m', '2024-07-18 06:30', '4.6', '742', '21']]
for bus_details_db in bus_details_extracted:
    cursor.execute(add_bus_details, bus_details_db)

db.commit()
cursor.close()
db.close()

print("rows inserted....")